# Model- 4 Deep Model

Exoplanet Exploration - Machine Learning Assignment
Summary
Create machine learning models capable of classifying candidate exoplanets from the raw dataset.

The algorithms used were:

* Model-1 - Logistic Refression
* Model-2 - Support Vector Machines (SVM) with Linear. Gaussian an dPolynomial kernels
* Model-3 - Random Forests
* Model-4 - Deep Model (Sequential)
* Model-5 - Decision Tree Model
Also, a hyper-parameter tuning with the meta-estimator function GridSearchCV() was used.

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\h_23m\anaconda3\envs\pythondata\lib\site-packages (0.0)


You should consider upgrading via the 'c:\users\h_23m\anaconda3\envs\pythondata\python.exe -m pip install --upgrade pip' command.


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

You should consider upgrading via the 'c:\users\h_23m\anaconda3\envs\pythondata\python.exe -m pip install --upgrade pip' command.


In [3]:
# Dependencies

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

### Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
pd.set_option('display.max_columns', None)
# Drop the null rows
df = df.dropna()
df.head(100)

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443,9.11,2.87,-1.62,25.8,2,5455,81,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638,39.30,31.04,-10.49,76.3,1,5853,158,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395,891.96,668.95,-230.35,505.6,1,5805,157,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406,926.16,874.33,-314.24,40.9,1,6031,169,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,0.762,0.139,-0.532,3.14020,0.06730,-0.06730,686.0,18.7,-18.7,2.77,0.90,-0.30,1160,427.65,420.33,-136.70,40.2,2,6046,189,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CONFIRMED,0,0,0,0,8.103633,2.030000e-05,-2.030000e-05,176.948600,0.002050,-0.002050,0.047,0.409,-0.047,3.16100,0.06350,-0.06350,887.9,24.5,-24.5,2.11,0.18,-0.20,687,52.82,16.30,-14.35,39.3,1,4861,131,-146,4.574,0.054,-0.041,0.732,0.062,-0.068,283.22141,40.421829,15.289
96,CONFIRMED,0,0,0,0,4.715108,1.940000e-05,-1.940000e-05,134.961780,0.003240,-0.003240,0.258,0.195,-0.258,2.50600,0.10300,-0.10300,417.5,22.0,-22.0,1.46,0.12,-0.14,823,108.84,33.58,-29.57,21.3,2,4861,131,-146,4.574,0.054,-0.041,0.732,0.062,-0.068,283.22141,40.421829,15.289
97,CONFIRMED,0,0,0,0,6.365840,8.060000e-06,-8.060000e-06,171.135750,0.001000,-0.001000,0.524,0.009,-0.460,2.97210,0.04080,-0.04080,780.7,13.1,-13.1,2.44,0.60,-0.21,914,165.33,121.73,-40.81,70.8,1,5776,155,-155,4.554,0.042,-0.168,0.849,0.212,-0.071,291.14951,40.420521,15.090
98,CONFIRMED,0,0,0,0,3.040330,3.370000e-07,-3.370000e-07,169.949011,0.000089,-0.000089,0.736,0.013,-0.024,1.84110,0.01680,-0.01680,10859.0,25.8,-25.8,8.99,0.80,-0.99,1021,257.29,86.59,-76.86,568.2,1,5035,151,-151,4.519,0.084,-0.063,0.791,0.071,-0.087,299.31610,40.822380,15.028


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6991 entries, 0 to 6990
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   object 
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_period         6991 non-null   float64
 6   koi_period_err1    6991 non-null   float64
 7   koi_period_err2    6991 non-null   float64
 8   koi_time0bk        6991 non-null   float64
 9   koi_time0bk_err1   6991 non-null   float64
 10  koi_time0bk_err2   6991 non-null   float64
 11  koi_impact         6991 non-null   float64
 12  koi_impact_err1    6991 non-null   float64
 13  koi_impact_err2    6991 non-null   float64
 14  koi_duration       6991 non-null   float64
 15  koi_duration_err1  6991 non-null   float64
 16  koi_duration_err2  6991 

In [7]:
df.describe()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6.991000e+03,6.991000e+03,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6.991000e+03,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6.991000e+03,6.991000e+03,6.991000e+03,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.00000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000
mean,0.157059,0.244743,0.202975,0.125018,56.191248,1.851122e-03,-1.851122e-03,164.488820,0.009340,-0.009340,0.730631,1.945368,-0.337243,5.549368,0.320715,-0.320715,2.391190e+04,130.789186,-130.789186,121.194193,19.483454,-40.352509,1095.152768,8.263769e+03,3.599918e+03,-4.302621e+03,276.038764,1.233729,5700.682449,143.898155,-161.206980,4.305049,0.121091,-0.14048,1.740749,0.352710,-0.388568,292.082406,43.812143,14.271508
std,0.363882,0.429966,0.402243,0.330763,117.570962,7.184503e-03,7.184503e-03,67.020475,0.021989,0.021989,3.346956,9.401850,1.349036,6.425205,0.665352,0.665352,8.241240e+04,4689.233030,4689.233030,3519.737626,436.752064,1366.939375,856.234809,1.775255e+05,5.607270e+04,9.924848e+04,824.788481,0.650089,802.060405,46.566511,71.448481,0.439238,0.132048,0.08199,5.903415,0.839017,1.907797,4.762908,3.606167,1.350802
min,0.000000,0.000000,0.000000,0.000000,0.259820,1.100000e-08,-1.568000e-01,120.515914,0.000009,-0.569000,0.000000,0.000000,-59.320000,0.167000,0.000000,-20.200000,7.500000e+00,0.000000,-388600.000000,0.140000,0.000000,-77180.000000,92.000000,2.000000e-02,0.000000e+00,-5.600031e+06,1.600000,1.000000,2661.000000,0.000000,-1733.000000,0.047000,0.000000,-1.00700,0.109000,0.000000,-103.825000,279.856080,36.577381,6.966000
25%,0.000000,0.000000,0.000000,0.000000,2.620126,5.005000e-06,-2.401000e-04,132.683917,0.001145,-0.010000,0.198000,0.039900,-0.446000,2.446000,0.048900,-0.334000,1.604000e+02,9.400000,-48.000000,1.400000,0.240000,-2.080000,554.000000,2.227000e+01,1.139000e+01,-3.068950e+02,12.400000,1.000000,5297.000000,105.000000,-197.000000,4.209000,0.044000,-0.19500,0.829000,0.128000,-0.252000,288.704730,40.797760,13.455000
50%,0.000000,0.000000,0.000000,0.000000,8.947426,3.300000e-05,-3.300000e-05,136.739230,0.003990,-0.003990,0.539000,0.189800,-0.204000,3.793000,0.137000,-0.137000,4.245000e+02,20.200000,-20.200000,2.420000,0.520000,-0.310000,888.000000,1.466700e+02,8.090000e+01,-4.443000e+01,23.700000,1.000000,5757.000000,156.000000,-159.000000,4.436000,0.070000,-0.12700,0.999000,0.248000,-0.111000,292.314760,43.679661,14.534000
75%,0.000000,0.000000,0.000000,0.000000,34.282605,2.401000e-04,-5.005000e-06,169.937005,0.010000,-0.001145,0.891000,0.379000,-0.044000,6.197500,0.334000,-0.048900,1.533250e+03,48.000000,-9.400000,15.965000,2.650000,-0.140000,1397.500000,9.006700e+02,5.529550e+02,-6.400000e+00,82.250000,1.000000,6110.000000,174.000000,-112.000000,4.543000,0.149000,-0.08800,1.357000,0.357000,-0.069000,295.888550,46.693659,15.322000
max,1.000000,1.000000,1.000000,1.000000,1071.232624,1.568000e-01,-1.100000e-08,1472.522306,0.569000,-0.000009,100.806000,85.540000,0.000000,138.540000,20.200000,0.000000,1.541400e+06,388600.000000,0.000000,200346.000000,21640.000000,0.000000,14667.000000,1.094755e+07,3.617133e+06,0.000000e+00,9054.700000,8.000000,15896.000000,676.000000,0.000000,5.364000,1.472000,0.00000,180.013000,25.956000,0.000000,301.720760,52.336010,19.065000


### Select your features (columns)

In [8]:
# Set features. This will also be used as your x values.
# selected_features = df[['names', 'of', 'selected', 'features', 'here']]

selected_features = df[['koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co','koi_fpflag_ec',
                        'koi_period','koi_period_err1','koi_period_err2',
                        'koi_time0bk','koi_time0bk_err1','koi_time0bk_err2',
                        'koi_impact','koi_impact_err1','koi_impact_err2',
                        'koi_duration','koi_duration_err1','koi_duration_err2',
                        'koi_depth','koi_depth_err1','koi_depth_err2',
                        'koi_prad','koi_prad_err1','koi_prad_err2',
                        'koi_teq','koi_insol','koi_insol_err1','koi_insol_err2',
                        'koi_model_snr','koi_steff','koi_steff_err1','koi_steff_err2',
                        'koi_slogg','koi_slogg_err1','koi_slogg_err2',
                        'koi_srad','koi_srad_err1','koi_srad_err2',
                        'ra','dec','koi_kepmag']]

feature_names = selected_features.columns
selected_features.head(20)

# OR

# # Set target, features and feature_names.
# target = df["koi_disposition"]  (y)
# data = df.drop("koi_disposition", axis=1)  (=X)
# feature_names = data.columns
# data.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.5860,0.0590,-0.4430,4.50700,0.11600,-0.11600,874.8,35.5,-35.5,2.83,0.32,-0.19,443,9.11,2.87,-1.62,25.8,5455,81,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,0.9690,5.1260,-0.0770,1.78220,0.03410,-0.03410,10829.0,171.0,-171.0,14.60,3.92,-1.31,638,39.30,31.04,-10.49,76.3,5853,158,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.2760,0.1150,-0.0920,2.40641,0.00537,-0.00537,8079.2,12.8,-12.8,33.46,8.50,-2.83,1395,891.96,668.95,-230.35,505.6,5805,157,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,0.7010,0.2350,-0.4780,1.65450,0.04200,-0.04200,603.3,16.9,-16.9,2.75,0.88,-0.35,1406,926.16,874.33,-314.24,40.9,6031,169,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,0.7620,0.1390,-0.5320,3.14020,0.06730,-0.06730,686.0,18.7,-18.7,2.77,0.90,-0.30,1160,427.65,420.33,-136.70,40.2,6046,189,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
5,0,0,0,0,2.566589,1.780000e-05,-1.780000e-05,179.554370,0.004610,-0.004610,0.7550,0.2120,-0.5230,2.42900,0.16500,-0.16500,226.5,16.8,-16.8,1.59,0.52,-0.17,1360,807.74,793.91,-258.20,15.0,6046,189,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
6,0,0,0,0,16.068647,1.090000e-05,-1.090000e-05,173.621937,0.000517,-0.000517,0.0520,0.2620,-0.0520,3.53470,0.02410,-0.02410,4914.3,33.3,-33.3,5.76,0.22,-0.49,600,30.75,4.46,-6.66,161.9,5031,75,-83,4.485,0.083,-0.028,0.848,0.033,-0.072,286.99948,48.375790,15.841
7,0,0,0,0,2.470613,2.700000e-08,-2.700000e-08,122.763305,0.000009,-0.000009,0.8180,0.0010,-0.0010,1.74319,0.00107,-0.00107,14231.0,4.2,-4.2,13.04,0.51,-0.51,1339,761.46,106.21,-95.76,4304.3,5820,78,-78,4.457,0.024,-0.024,0.964,0.038,-0.038,286.80847,49.316399,11.338
8,0,1,0,0,2.204735,4.300000e-08,-4.300000e-08,121.358542,0.000016,-0.000016,0.2240,0.1590,-0.2160,3.88864,0.00203,-0.00203,6674.7,1.7,-1.7,16.10,0.81,-0.91,2048,4148.92,651.59,-654.46,5945.9,6440,76,-89,4.019,0.033,-0.027,1.952,0.099,-0.110,292.24728,47.969521,10.463
9,0,0,0,0,3.522498,1.980000e-07,-1.980000e-07,121.119423,0.000047,-0.000047,0.6310,0.0070,-0.0070,3.19843,0.00653,-0.00653,9145.7,6.6,-6.6,14.59,1.11,-1.11,1521,1264.67,307.23,-276.48,1741.5,6225,112,-137,4.169,0.055,-0.045,1.451,0.110,-0.110,281.28812,42.451080,13.563


In [9]:
X = selected_features.values
print(X.shape)

(6991, 39)


### Create a Train Test Split

Use `koi_disposition` for the y values

In [10]:

# get X and y variables
y = df["koi_disposition"].values  # target
X = selected_features    # data

In [11]:

df["koi_disposition"].value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [12]:
target_labels = ["FALSE POSITIVE", "CONFIRMED", "CANDIDATE"]  
target_labels 

['FALSE POSITIVE', 'CONFIRMED', 'CANDIDATE']

In [13]:
# Label-encode data set
# Scale your data using MinMaxScaler
# Import dependencies
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from tensorflow.keras.utils import to_categorical

# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)
print(encoded_y)

# One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y
print(one_hot_y)

[1 2 2 ... 0 2 2]
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [14]:
# split the data to train and test values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_y, random_state=42, stratify=y)

In [15]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6080,1,0,0,0,12.496435,2.213000e-04,-2.213000e-04,132.035800,0.014300,-0.014300,1.1700,0.078,-0.0850,84.32000,1.2000,-1.2000,271.7,2.5,-2.5,55.34,10.84,-25.30,1397,899.44,536.52,-676.12,141.5,6821,190,-286,3.805,0.390,-0.130,2.730,0.535,-1.248,289.23080,44.412483,13.054
3001,0,0,0,0,11.615625,1.528000e-04,-1.528000e-04,131.968430,0.008230,-0.008230,0.9770,0.007,-0.7050,2.23300,0.3620,-0.3620,191.2,28.0,-28.0,2.84,0.43,-1.00,905,158.69,65.40,-95.39,8.8,5332,88,-72,4.083,0.368,-0.092,1.453,0.218,-0.510,293.52756,41.111439,15.162
570,0,1,0,0,10.980246,6.930000e-07,-6.930000e-07,137.137607,0.000053,-0.000053,0.7330,0.001,-0.0010,3.74464,0.0034,-0.0034,50078.0,44.7,-44.7,21.94,5.82,-2.91,821,107.47,83.74,-34.92,1555.4,5952,159,-159,4.462,0.098,-0.182,0.897,0.238,-0.119,282.79764,43.578129,14.212
4897,1,0,0,0,466.908240,1.194000e-02,-1.194000e-02,136.373100,0.019000,-0.019000,0.0868,0.433,-0.0867,2.64000,0.6930,-0.6930,660.0,183.0,-183.0,2.19,1.13,-0.26,210,0.46,0.76,-0.14,5.4,5340,185,-146,4.456,0.102,-0.361,0.867,0.448,-0.103,297.65436,43.178551,15.202
625,0,1,1,1,1.061933,1.250000e-06,-1.250000e-06,133.850441,0.000978,-0.000978,0.7130,0.112,-0.2160,2.14290,0.0386,-0.0386,133.6,2.0,-2.0,2.29,0.48,-0.71,2508,9391.15,5840.09,-5394.87,80.2,6134,167,-167,3.975,0.259,-0.111,1.851,0.383,-0.575,288.90253,44.632992,12.953


### Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [16]:
# Scale your data using MinMaxScaler
# Import dependencies
from sklearn.preprocessing import MinMaxScaler

# scale the data
X_minmax = MinMaxScaler().fit(X_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)

# no need to scale y since it is already categorical
y_train

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

### Train the Model (Deep Model)



In [17]:

# define the model
from tensorflow.keras.models import Sequential
model = Sequential()

from tensorflow.keras.layers import Dense
number_inputs = 39
number_hidden_nodes = 6
model.add(Dense(units=number_hidden_nodes, activation='relu', input_dim=number_inputs))
number_classes = 3
model.add(Dense(units=number_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 240       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 21        
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
# Train the model
model.fit(X_train_minmax, y_train, epochs=1000, shuffle=True, verbose=2)

Epoch 1/1000
164/164 - 0s - loss: 1.0597 - accuracy: 0.4314
Epoch 2/1000
164/164 - 0s - loss: 0.8643 - accuracy: 0.5113
Epoch 3/1000
164/164 - 0s - loss: 0.7113 - accuracy: 0.6824
Epoch 4/1000
164/164 - 0s - loss: 0.5850 - accuracy: 0.7337
Epoch 5/1000
164/164 - 0s - loss: 0.5068 - accuracy: 0.7496
Epoch 6/1000
164/164 - 0s - loss: 0.4605 - accuracy: 0.7946
Epoch 7/1000
164/164 - 0s - loss: 0.4327 - accuracy: 0.8020
Epoch 8/1000
164/164 - 0s - loss: 0.4152 - accuracy: 0.8085
Epoch 9/1000
164/164 - 0s - loss: 0.4029 - accuracy: 0.8095
Epoch 10/1000
164/164 - 0s - loss: 0.3937 - accuracy: 0.8146
Epoch 11/1000
164/164 - 0s - loss: 0.3870 - accuracy: 0.8171
Epoch 12/1000
164/164 - 0s - loss: 0.3806 - accuracy: 0.8222
Epoch 13/1000
164/164 - 0s - loss: 0.3733 - accuracy: 0.8325
Epoch 14/1000
164/164 - 0s - loss: 0.3683 - accuracy: 0.8280
Epoch 15/1000
164/164 - 0s - loss: 0.3630 - accuracy: 0.8364
Epoch 16/1000
164/164 - 0s - loss: 0.3581 - accuracy: 0.8343
Epoch 17/1000
164/164 - 0s - loss

Epoch 135/1000
164/164 - 0s - loss: 0.2881 - accuracy: 0.8835
Epoch 136/1000
164/164 - 0s - loss: 0.2896 - accuracy: 0.8827
Epoch 137/1000
164/164 - 0s - loss: 0.2877 - accuracy: 0.8806
Epoch 138/1000
164/164 - 0s - loss: 0.2885 - accuracy: 0.8806
Epoch 139/1000
164/164 - 0s - loss: 0.2880 - accuracy: 0.8808
Epoch 140/1000
164/164 - 0s - loss: 0.2888 - accuracy: 0.8781
Epoch 141/1000
164/164 - 0s - loss: 0.2897 - accuracy: 0.8791
Epoch 142/1000
164/164 - 0s - loss: 0.2885 - accuracy: 0.8837
Epoch 143/1000
164/164 - 0s - loss: 0.2883 - accuracy: 0.8827
Epoch 144/1000
164/164 - 0s - loss: 0.2891 - accuracy: 0.8821
Epoch 145/1000
164/164 - 0s - loss: 0.2876 - accuracy: 0.8817
Epoch 146/1000
164/164 - 0s - loss: 0.2869 - accuracy: 0.8819
Epoch 147/1000
164/164 - 0s - loss: 0.2892 - accuracy: 0.8789
Epoch 148/1000
164/164 - 0s - loss: 0.2873 - accuracy: 0.8825
Epoch 149/1000
164/164 - 0s - loss: 0.2873 - accuracy: 0.8816
Epoch 150/1000
164/164 - 0s - loss: 0.2869 - accuracy: 0.8785
Epoch 15

164/164 - 0s - loss: 0.2778 - accuracy: 0.8858
Epoch 268/1000
164/164 - 0s - loss: 0.2779 - accuracy: 0.8859
Epoch 269/1000
164/164 - 0s - loss: 0.2774 - accuracy: 0.8850
Epoch 270/1000
164/164 - 0s - loss: 0.2761 - accuracy: 0.8871
Epoch 271/1000
164/164 - 0s - loss: 0.2784 - accuracy: 0.8869
Epoch 272/1000
164/164 - 0s - loss: 0.2788 - accuracy: 0.8835
Epoch 273/1000
164/164 - 0s - loss: 0.2763 - accuracy: 0.8896
Epoch 274/1000
164/164 - 0s - loss: 0.2774 - accuracy: 0.8861
Epoch 275/1000
164/164 - 0s - loss: 0.2755 - accuracy: 0.8873
Epoch 276/1000
164/164 - 0s - loss: 0.2776 - accuracy: 0.8879
Epoch 277/1000
164/164 - 0s - loss: 0.2752 - accuracy: 0.8880
Epoch 278/1000
164/164 - 0s - loss: 0.2774 - accuracy: 0.8852
Epoch 279/1000
164/164 - 0s - loss: 0.2772 - accuracy: 0.8865
Epoch 280/1000
164/164 - 0s - loss: 0.2756 - accuracy: 0.8867
Epoch 281/1000
164/164 - 0s - loss: 0.2777 - accuracy: 0.8852
Epoch 282/1000
164/164 - 0s - loss: 0.2782 - accuracy: 0.8819
Epoch 283/1000
164/164 

Epoch 400/1000
164/164 - 0s - loss: 0.2644 - accuracy: 0.8911
Epoch 401/1000
164/164 - 0s - loss: 0.2657 - accuracy: 0.8896
Epoch 402/1000
164/164 - 0s - loss: 0.2652 - accuracy: 0.8919
Epoch 403/1000
164/164 - 0s - loss: 0.2655 - accuracy: 0.8896
Epoch 404/1000
164/164 - 0s - loss: 0.2665 - accuracy: 0.8915
Epoch 405/1000
164/164 - 0s - loss: 0.2644 - accuracy: 0.8924
Epoch 406/1000
164/164 - 0s - loss: 0.2664 - accuracy: 0.8920
Epoch 407/1000
164/164 - 0s - loss: 0.2662 - accuracy: 0.8907
Epoch 408/1000
164/164 - 0s - loss: 0.2647 - accuracy: 0.8888
Epoch 409/1000
164/164 - 0s - loss: 0.2671 - accuracy: 0.8865
Epoch 410/1000
164/164 - 0s - loss: 0.2638 - accuracy: 0.8905
Epoch 411/1000
164/164 - 0s - loss: 0.2656 - accuracy: 0.8894
Epoch 412/1000
164/164 - 0s - loss: 0.2653 - accuracy: 0.8880
Epoch 413/1000
164/164 - 0s - loss: 0.2659 - accuracy: 0.8922
Epoch 414/1000
164/164 - 0s - loss: 0.2648 - accuracy: 0.8879
Epoch 415/1000
164/164 - 0s - loss: 0.2657 - accuracy: 0.8898
Epoch 41

Epoch 533/1000
164/164 - 0s - loss: 0.2600 - accuracy: 0.8915
Epoch 534/1000
164/164 - 0s - loss: 0.2601 - accuracy: 0.8920
Epoch 535/1000
164/164 - 0s - loss: 0.2604 - accuracy: 0.8924
Epoch 536/1000
164/164 - 0s - loss: 0.2593 - accuracy: 0.8917
Epoch 537/1000
164/164 - 0s - loss: 0.2597 - accuracy: 0.8907
Epoch 538/1000
164/164 - 0s - loss: 0.2590 - accuracy: 0.8932
Epoch 539/1000
164/164 - 0s - loss: 0.2593 - accuracy: 0.8911
Epoch 540/1000
164/164 - 0s - loss: 0.2612 - accuracy: 0.8888
Epoch 541/1000
164/164 - 0s - loss: 0.2604 - accuracy: 0.8907
Epoch 542/1000
164/164 - 0s - loss: 0.2599 - accuracy: 0.8934
Epoch 543/1000
164/164 - 0s - loss: 0.2599 - accuracy: 0.8911
Epoch 544/1000
164/164 - 0s - loss: 0.2602 - accuracy: 0.8917
Epoch 545/1000
164/164 - 0s - loss: 0.2597 - accuracy: 0.8920
Epoch 546/1000
164/164 - 0s - loss: 0.2601 - accuracy: 0.8903
Epoch 547/1000
164/164 - 0s - loss: 0.2591 - accuracy: 0.8924
Epoch 548/1000
164/164 - 0s - loss: 0.2591 - accuracy: 0.8919
Epoch 54

164/164 - 0s - loss: 0.2575 - accuracy: 0.8926
Epoch 666/1000
164/164 - 0s - loss: 0.2565 - accuracy: 0.8941
Epoch 667/1000
164/164 - 0s - loss: 0.2576 - accuracy: 0.8922
Epoch 668/1000
164/164 - 0s - loss: 0.2577 - accuracy: 0.8932
Epoch 669/1000
164/164 - 0s - loss: 0.2564 - accuracy: 0.8924
Epoch 670/1000
164/164 - 0s - loss: 0.2567 - accuracy: 0.8936
Epoch 671/1000
164/164 - 0s - loss: 0.2570 - accuracy: 0.8941
Epoch 672/1000
164/164 - 0s - loss: 0.2575 - accuracy: 0.8928
Epoch 673/1000
164/164 - 0s - loss: 0.2566 - accuracy: 0.8919
Epoch 674/1000
164/164 - 0s - loss: 0.2566 - accuracy: 0.8945
Epoch 675/1000
164/164 - 0s - loss: 0.2591 - accuracy: 0.8928
Epoch 676/1000
164/164 - 0s - loss: 0.2577 - accuracy: 0.8941
Epoch 677/1000
164/164 - 0s - loss: 0.2577 - accuracy: 0.8913
Epoch 678/1000
164/164 - 0s - loss: 0.2576 - accuracy: 0.8932
Epoch 679/1000
164/164 - 0s - loss: 0.2579 - accuracy: 0.8909
Epoch 680/1000
164/164 - 0s - loss: 0.2582 - accuracy: 0.8899
Epoch 681/1000
164/164 

Epoch 798/1000
164/164 - 0s - loss: 0.2589 - accuracy: 0.8930
Epoch 799/1000
164/164 - 0s - loss: 0.2578 - accuracy: 0.8922
Epoch 800/1000
164/164 - 0s - loss: 0.2572 - accuracy: 0.8945
Epoch 801/1000
164/164 - 0s - loss: 0.2562 - accuracy: 0.8926
Epoch 802/1000
164/164 - 0s - loss: 0.2572 - accuracy: 0.8919
Epoch 803/1000
164/164 - 0s - loss: 0.2564 - accuracy: 0.8936
Epoch 804/1000
164/164 - 0s - loss: 0.2565 - accuracy: 0.8957
Epoch 805/1000
164/164 - 0s - loss: 0.2573 - accuracy: 0.8898
Epoch 806/1000
164/164 - 0s - loss: 0.2573 - accuracy: 0.8913
Epoch 807/1000
164/164 - 0s - loss: 0.2573 - accuracy: 0.8936
Epoch 808/1000
164/164 - 0s - loss: 0.2573 - accuracy: 0.8911
Epoch 809/1000
164/164 - 0s - loss: 0.2557 - accuracy: 0.8930
Epoch 810/1000
164/164 - 0s - loss: 0.2568 - accuracy: 0.8957
Epoch 811/1000
164/164 - 0s - loss: 0.2563 - accuracy: 0.8932
Epoch 812/1000
164/164 - 0s - loss: 0.2549 - accuracy: 0.8940
Epoch 813/1000
164/164 - 0s - loss: 0.2586 - accuracy: 0.8911
Epoch 81

Epoch 931/1000
164/164 - 0s - loss: 0.2557 - accuracy: 0.8951
Epoch 932/1000
164/164 - 0s - loss: 0.2579 - accuracy: 0.8926
Epoch 933/1000
164/164 - 0s - loss: 0.2576 - accuracy: 0.8920
Epoch 934/1000
164/164 - 0s - loss: 0.2542 - accuracy: 0.8930
Epoch 935/1000
164/164 - 0s - loss: 0.2568 - accuracy: 0.8892
Epoch 936/1000
164/164 - 0s - loss: 0.2557 - accuracy: 0.8938
Epoch 937/1000
164/164 - 0s - loss: 0.2561 - accuracy: 0.8947
Epoch 938/1000
164/164 - 0s - loss: 0.2555 - accuracy: 0.8947
Epoch 939/1000
164/164 - 0s - loss: 0.2579 - accuracy: 0.8907
Epoch 940/1000
164/164 - 0s - loss: 0.2549 - accuracy: 0.8943
Epoch 941/1000
164/164 - 0s - loss: 0.2570 - accuracy: 0.8915
Epoch 942/1000
164/164 - 0s - loss: 0.2556 - accuracy: 0.8940
Epoch 943/1000
164/164 - 0s - loss: 0.2578 - accuracy: 0.8940
Epoch 944/1000
164/164 - 0s - loss: 0.2559 - accuracy: 0.8913
Epoch 945/1000
164/164 - 0s - loss: 0.2543 - accuracy: 0.8934
Epoch 946/1000
164/164 - 0s - loss: 0.2555 - accuracy: 0.8951
Epoch 94

In [20]:
model_loss, model_accuracy = model.evaluate(X_test_minmax, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


55/55 - 0s - loss: 0.2416 - accuracy: 0.8987
Loss: 0.2415650337934494, Accuracy: 0.8987414240837097


In [21]:
# "Sequential" object has no attribute 'score' so no data score
# print(f"Training Data Score: {model.score(X_train_minmax, y_train)}")
# print(f"Testing Data Score: {model.score(X_test_minmax, y_test)}")

In [22]:

# Deep learning
deep_model = Sequential()
deep_model.add(Dense(units=6, activation='relu', input_dim=number_inputs))
deep_model.add(Dense(units=6, activation='relu'))
deep_model.add(Dense(units=number_classes, activation='softmax'))
deep_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6)                 240       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 21        
Total params: 303
Trainable params: 303
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the Model
deep_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:

# Train the model
deep_model.fit(X_train_minmax, y_train, epochs=1000, shuffle=True, verbose=2)

Epoch 1/1000
164/164 - 0s - loss: 0.8635 - accuracy: 0.5880
Epoch 2/1000
164/164 - 0s - loss: 0.5917 - accuracy: 0.7534
Epoch 3/1000
164/164 - 0s - loss: 0.4569 - accuracy: 0.7809
Epoch 4/1000
164/164 - 0s - loss: 0.4114 - accuracy: 0.8024
Epoch 5/1000
164/164 - 0s - loss: 0.3945 - accuracy: 0.8072
Epoch 6/1000
164/164 - 0s - loss: 0.3864 - accuracy: 0.8116
Epoch 7/1000
164/164 - 0s - loss: 0.3788 - accuracy: 0.8163
Epoch 8/1000
164/164 - 0s - loss: 0.3751 - accuracy: 0.8205
Epoch 9/1000
164/164 - 0s - loss: 0.3724 - accuracy: 0.8177
Epoch 10/1000
164/164 - 0s - loss: 0.3662 - accuracy: 0.8217
Epoch 11/1000
164/164 - 0s - loss: 0.3629 - accuracy: 0.8253
Epoch 12/1000
164/164 - 0s - loss: 0.3606 - accuracy: 0.8226
Epoch 13/1000
164/164 - 0s - loss: 0.3574 - accuracy: 0.8325
Epoch 14/1000
164/164 - 0s - loss: 0.3533 - accuracy: 0.8323
Epoch 15/1000
164/164 - 0s - loss: 0.3514 - accuracy: 0.8333
Epoch 16/1000
164/164 - 0s - loss: 0.3493 - accuracy: 0.8320
Epoch 17/1000
164/164 - 0s - loss

Epoch 135/1000
164/164 - 0s - loss: 0.2773 - accuracy: 0.8840
Epoch 136/1000
164/164 - 0s - loss: 0.2781 - accuracy: 0.8831
Epoch 137/1000
164/164 - 0s - loss: 0.2789 - accuracy: 0.8806
Epoch 138/1000
164/164 - 0s - loss: 0.2735 - accuracy: 0.8844
Epoch 139/1000
164/164 - 0s - loss: 0.2749 - accuracy: 0.8846
Epoch 140/1000
164/164 - 0s - loss: 0.2796 - accuracy: 0.8787
Epoch 141/1000
164/164 - 0s - loss: 0.2767 - accuracy: 0.8833
Epoch 142/1000
164/164 - 0s - loss: 0.2783 - accuracy: 0.8808
Epoch 143/1000
164/164 - 0s - loss: 0.2769 - accuracy: 0.8814
Epoch 144/1000
164/164 - 0s - loss: 0.2765 - accuracy: 0.8838
Epoch 145/1000
164/164 - 0s - loss: 0.2750 - accuracy: 0.8846
Epoch 146/1000
164/164 - 0s - loss: 0.2761 - accuracy: 0.8821
Epoch 147/1000
164/164 - 0s - loss: 0.2752 - accuracy: 0.8787
Epoch 148/1000
164/164 - 0s - loss: 0.2768 - accuracy: 0.8804
Epoch 149/1000
164/164 - 0s - loss: 0.2761 - accuracy: 0.8802
Epoch 150/1000
164/164 - 0s - loss: 0.2739 - accuracy: 0.8838
Epoch 15

Epoch 268/1000
164/164 - 0s - loss: 0.2624 - accuracy: 0.8890
Epoch 269/1000
164/164 - 0s - loss: 0.2626 - accuracy: 0.8886
Epoch 270/1000
164/164 - 0s - loss: 0.2597 - accuracy: 0.8884
Epoch 271/1000
164/164 - 0s - loss: 0.2602 - accuracy: 0.8898
Epoch 272/1000
164/164 - 0s - loss: 0.2592 - accuracy: 0.8879
Epoch 273/1000
164/164 - 0s - loss: 0.2605 - accuracy: 0.8869
Epoch 274/1000
164/164 - 0s - loss: 0.2606 - accuracy: 0.8884
Epoch 275/1000
164/164 - 0s - loss: 0.2591 - accuracy: 0.8890
Epoch 276/1000
164/164 - 0s - loss: 0.2621 - accuracy: 0.8877
Epoch 277/1000
164/164 - 0s - loss: 0.2591 - accuracy: 0.8886
Epoch 278/1000
164/164 - 0s - loss: 0.2601 - accuracy: 0.8877
Epoch 279/1000
164/164 - 0s - loss: 0.2598 - accuracy: 0.8886
Epoch 280/1000
164/164 - 0s - loss: 0.2598 - accuracy: 0.8890
Epoch 281/1000
164/164 - 0s - loss: 0.2601 - accuracy: 0.8894
Epoch 282/1000
164/164 - 0s - loss: 0.2570 - accuracy: 0.8924
Epoch 283/1000
164/164 - 0s - loss: 0.2595 - accuracy: 0.8880
Epoch 28

164/164 - 0s - loss: 0.2507 - accuracy: 0.8920
Epoch 401/1000
164/164 - 0s - loss: 0.2490 - accuracy: 0.8938
Epoch 402/1000
164/164 - 0s - loss: 0.2526 - accuracy: 0.8924
Epoch 403/1000
164/164 - 0s - loss: 0.2517 - accuracy: 0.8922
Epoch 404/1000
164/164 - 0s - loss: 0.2533 - accuracy: 0.8909
Epoch 405/1000
164/164 - 0s - loss: 0.2516 - accuracy: 0.8920
Epoch 406/1000
164/164 - 0s - loss: 0.2509 - accuracy: 0.8901
Epoch 407/1000
164/164 - 0s - loss: 0.2506 - accuracy: 0.8920
Epoch 408/1000
164/164 - 0s - loss: 0.2523 - accuracy: 0.8911
Epoch 409/1000
164/164 - 0s - loss: 0.2505 - accuracy: 0.8909
Epoch 410/1000
164/164 - 0s - loss: 0.2503 - accuracy: 0.8905
Epoch 411/1000
164/164 - 0s - loss: 0.2493 - accuracy: 0.8947
Epoch 412/1000
164/164 - 0s - loss: 0.2521 - accuracy: 0.8936
Epoch 413/1000
164/164 - 0s - loss: 0.2515 - accuracy: 0.8920
Epoch 414/1000
164/164 - 0s - loss: 0.2530 - accuracy: 0.8943
Epoch 415/1000
164/164 - 0s - loss: 0.2522 - accuracy: 0.8922
Epoch 416/1000
164/164 

Epoch 533/1000
164/164 - 0s - loss: 0.2512 - accuracy: 0.8911
Epoch 534/1000
164/164 - 0s - loss: 0.2496 - accuracy: 0.8917
Epoch 535/1000
164/164 - 0s - loss: 0.2489 - accuracy: 0.8940
Epoch 536/1000
164/164 - 0s - loss: 0.2459 - accuracy: 0.8972
Epoch 537/1000
164/164 - 0s - loss: 0.2480 - accuracy: 0.8920
Epoch 538/1000
164/164 - 0s - loss: 0.2469 - accuracy: 0.8955
Epoch 539/1000
164/164 - 0s - loss: 0.2475 - accuracy: 0.8917
Epoch 540/1000
164/164 - 0s - loss: 0.2470 - accuracy: 0.8957
Epoch 541/1000
164/164 - 0s - loss: 0.2487 - accuracy: 0.8951
Epoch 542/1000
164/164 - 0s - loss: 0.2463 - accuracy: 0.8909
Epoch 543/1000
164/164 - 0s - loss: 0.2473 - accuracy: 0.8924
Epoch 544/1000
164/164 - 0s - loss: 0.2491 - accuracy: 0.8945
Epoch 545/1000
164/164 - 0s - loss: 0.2495 - accuracy: 0.8928
Epoch 546/1000
164/164 - 0s - loss: 0.2491 - accuracy: 0.8899
Epoch 547/1000
164/164 - 0s - loss: 0.2475 - accuracy: 0.8898
Epoch 548/1000
164/164 - 0s - loss: 0.2491 - accuracy: 0.8894
Epoch 54

Epoch 666/1000
164/164 - 0s - loss: 0.2460 - accuracy: 0.8932
Epoch 667/1000
164/164 - 0s - loss: 0.2440 - accuracy: 0.8947
Epoch 668/1000
164/164 - 0s - loss: 0.2482 - accuracy: 0.8915
Epoch 669/1000
164/164 - 0s - loss: 0.2466 - accuracy: 0.8919
Epoch 670/1000
164/164 - 0s - loss: 0.2447 - accuracy: 0.8945
Epoch 671/1000
164/164 - 0s - loss: 0.2443 - accuracy: 0.8964
Epoch 672/1000
164/164 - 0s - loss: 0.2442 - accuracy: 0.8961
Epoch 673/1000
164/164 - 0s - loss: 0.2449 - accuracy: 0.8964
Epoch 674/1000
164/164 - 0s - loss: 0.2471 - accuracy: 0.8928
Epoch 675/1000
164/164 - 0s - loss: 0.2434 - accuracy: 0.8959
Epoch 676/1000
164/164 - 0s - loss: 0.2451 - accuracy: 0.8940
Epoch 677/1000
164/164 - 0s - loss: 0.2438 - accuracy: 0.8966
Epoch 678/1000
164/164 - 0s - loss: 0.2456 - accuracy: 0.8936
Epoch 679/1000
164/164 - 0s - loss: 0.2436 - accuracy: 0.8947
Epoch 680/1000
164/164 - 0s - loss: 0.2442 - accuracy: 0.8949
Epoch 681/1000
164/164 - 0s - loss: 0.2447 - accuracy: 0.8962
Epoch 68

Epoch 799/1000
164/164 - 0s - loss: 0.2402 - accuracy: 0.8989
Epoch 800/1000
164/164 - 0s - loss: 0.2441 - accuracy: 0.8934
Epoch 801/1000
164/164 - 0s - loss: 0.2418 - accuracy: 0.8972
Epoch 802/1000
164/164 - 0s - loss: 0.2426 - accuracy: 0.8980
Epoch 803/1000
164/164 - 0s - loss: 0.2423 - accuracy: 0.8962
Epoch 804/1000
164/164 - 0s - loss: 0.2399 - accuracy: 0.8980
Epoch 805/1000
164/164 - 0s - loss: 0.2408 - accuracy: 0.9004
Epoch 806/1000
164/164 - 0s - loss: 0.2420 - accuracy: 0.8970
Epoch 807/1000
164/164 - 0s - loss: 0.2403 - accuracy: 0.8962
Epoch 808/1000
164/164 - 0s - loss: 0.2426 - accuracy: 0.8959
Epoch 809/1000
164/164 - 0s - loss: 0.2434 - accuracy: 0.8945
Epoch 810/1000
164/164 - 0s - loss: 0.2432 - accuracy: 0.8989
Epoch 811/1000
164/164 - 0s - loss: 0.2416 - accuracy: 0.8970
Epoch 812/1000
164/164 - 0s - loss: 0.2394 - accuracy: 0.8985
Epoch 813/1000
164/164 - 0s - loss: 0.2467 - accuracy: 0.8947
Epoch 814/1000
164/164 - 0s - loss: 0.2397 - accuracy: 0.8976
Epoch 81

164/164 - 0s - loss: 0.2377 - accuracy: 0.9001
Epoch 932/1000
164/164 - 0s - loss: 0.2426 - accuracy: 0.8957
Epoch 933/1000
164/164 - 0s - loss: 0.2412 - accuracy: 0.8968
Epoch 934/1000
164/164 - 0s - loss: 0.2401 - accuracy: 0.9001
Epoch 935/1000
164/164 - 0s - loss: 0.2398 - accuracy: 0.9004
Epoch 936/1000
164/164 - 0s - loss: 0.2405 - accuracy: 0.8957
Epoch 937/1000
164/164 - 0s - loss: 0.2389 - accuracy: 0.8989
Epoch 938/1000
164/164 - 0s - loss: 0.2437 - accuracy: 0.8962
Epoch 939/1000
164/164 - 0s - loss: 0.2419 - accuracy: 0.8987
Epoch 940/1000
164/164 - 0s - loss: 0.2381 - accuracy: 0.8991
Epoch 941/1000
164/164 - 0s - loss: 0.2395 - accuracy: 0.8987
Epoch 942/1000
164/164 - 0s - loss: 0.2392 - accuracy: 0.8981
Epoch 943/1000
164/164 - 0s - loss: 0.2399 - accuracy: 0.8980
Epoch 944/1000
164/164 - 0s - loss: 0.2386 - accuracy: 0.8999
Epoch 945/1000
164/164 - 0s - loss: 0.2393 - accuracy: 0.8962
Epoch 946/1000
164/164 - 0s - loss: 0.2391 - accuracy: 0.8983
Epoch 947/1000
164/164 

In [25]:

deep_model_loss, deep_model_accuracy = deep_model.evaluate(X_test_minmax, y_test, verbose=2)
print(f"Loss: {deep_model_loss}, Accuracy: {deep_model_accuracy}")

55/55 - 0s - loss: 0.2365 - accuracy: 0.8879
Loss: 0.23645108938217163, Accuracy: 0.8878718614578247


In [26]:
model.save("deep_model.h5")

In [27]:

# Convert the results to a Pandas data frame and make a short view of the results
# cvrf_results = pd.DataFrame(grid.cv_results_)
# cvrf_results_tiny = cvrf_results[['params', 'mean_test_score', 'std_test_score', 'rank_test_score', 'mean_train_score', 'std_train_score']]
# cvrf_results_tiny.sort_values(by='mean_test_score', ascending=False).head()